In [44]:
# importing LangChain library to bring together the different components needed for our model

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnablePassthrough , RunnableParallel
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.runnables import RunnableLambda
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.vectorstores import Qdrant
from qdrant_client import models, QdrantClient
import qdrant_client

In [ ]:
# API keys

import os
import dotenv

os.environ["OPENAI_API_KEY"]

In [83]:
# create a qdrant instance

os.environ['QDRANT_HOST']
os.environ["QDRANT_API_KEY"]

client = qdrant_client.QdrantClient(
    os.getenv("QDRANT_HOST"),
    api_key=os.getenv("QDRANT_API_KEY")
)

In [84]:
# create a qdrant instance

os.environ['QDRANT_HOST'] 
os.environ["QDRANT_API_KEY"]

client = qdrant_client.QdrantClient(
    os.getenv("QDRANT_HOST"),
    api_key=os.getenv("QDRANT_API_KEY")
)

In [85]:
# read the file

doc ="personas.txt"
data=""

with open(doc,'r') as f:
    data = f.read()

In [86]:
# split the text into chunks
#create a function to return chunks
def get_chunks(text):
    text_splitter=CharacterTextSplitter(
        separator= "\n",
        chunk_size=1000,
        chunk_overlap=100, # second chunk start  character from 800, overlap is used to stop loosing chunk
        length_function=len
    )

    chunks=text_splitter.split_text(text)
    return chunks

In [87]:
# get the chunks for the data
texts=get_chunks(data)

In [ ]:
# creating a new collection

vectors_config=models.VectorParams(
    # depends on model, we can google dimension. 1536 for openai
    # we are using openai embedding, for that size is 1536
    size=1536,
    distance=models.Distance.COSINE)

client.create_collection(
    collection_name="Personas",
    vectors_config=vectors_config,
)

In [89]:
# create a vector store object using langchain

# if we want to use any other embedding, we need to change size

os.environ["OPENAI_API_KEY"]

embeddings = OpenAIEmbeddings()

vector_store = Qdrant(
    client=client,
    collection_name="Personas",
    embeddings=embeddings,
)


In [90]:
# add chunks to vector store
vector_store.add_texts(texts)

['de48b50853cb4a8e867154cc4eeb861a',
 '98bd39867d114e11a945dec5b51ca83c',
 'a9aaaf71eb7c41e79b4ce92b26ed9a09',
 '17dae18c588a414782730a3650f8981b',
 'f720344887944471ba035a21b2666cf9',
 'cfa4f849c0aa4338a487828dfd26e756',
 '7fbb922bb5f242c9a9e16eaa3d6e8a74',
 '307aeb470de74fa8b6e1347ddaabca2f',
 '3563c9b55a8f42309689b017f65dfff7',
 'c152f57ad1984a88bf8fc7b75158aa43',
 '5a5e8338023341d88231c4bf17d15039',
 'fe7923b7dcb64a5cafc515118e1c2ad9',
 '6dd0867acb0c4ec0aa34f1c243702ffe',
 '28d60efddec542bdb05b65f6e96fdebb',
 '94ecb12f36e847c290dadc1f02091d33',
 '76830301cffe458eaff610676d1a607e',
 'f418d3f73dc44794a3b3f53ba20a191a',
 'f68a6c3767d9453c8e4abda299e9a929',
 '24b85720e3e741bab0843e605a1fef18',
 'a1cc2abca06f493bab6c94c89cdc1c9b',
 '185e043ba5f248a1a3299a780c1bc6a6',
 '90955689aa65493195a4efeb5cbf93f4',
 '68527d38e86d406ab17885cedec7bf48',
 'f2589bf1a07c42c7a270ae3627ae090f',
 '8cc35d6d0f134c2dbcc824da23d13dc2',
 '59e910b97eaf4953ad92eb5c2a5eccc7',
 'c58e66a4ce5f41ec8bbd500ddf446f7e',
 

In [91]:
# create retriever & let's try passing a question through to a these vectorstore objects


retriever = vector_store.as_retriever()

template1 = """"You are an strategizing AI agent for projects 
you provide insights which are categorized based on the personas provided to you. 
Your insights should reflect proper industry experience of the persona
 Please include the personas employed in the team composition.
     {context}

Question: {question}
"""

In [92]:
template2 = """"You are a AI smart assistant here to offer guidance and advice on a variety of tasks and queries. 
Utilize the available tools to move forward in finding the answer to your question
{context}

Question: {question}"""

In [93]:
template = template1 + template2

In [94]:
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

In [95]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [96]:
# invoke response

chain.invoke("How can I set up a project for developing a industrial applications, including needed skills, team members, and budget considerations?")

'To set up a project for developing industrial applications, you will need a team with a diverse set of skills and expertise. Here are some recommendations based on the personas provided:\n\n1. **Project Manager Maya**: As a certified Project Management Professional with experience in leading complex IT infrastructure projects, Maya can take on the role of project manager for this industrial applications project. She will be responsible for coordinating cross-functional teams, managing resources, and ensuring the project is delivered on time and within budget.\n\n2. **AI Prompt Engineer**: With a background in practical applications and a deep understanding of how systems operate and evolve, the AI Prompt Engineer can contribute valuable insights on the technical aspects of developing industrial applications. They can help in designing the architecture, implementing AI solutions, and optimizing the performance of the applications.\n\n3. **Product Manager Marco**: With expertise in mana

In [97]:
# stream response

for chunk in chain.stream(
"How can I set up a project for developing a industrial applications, including needed skills, team members, and budget considerations?"):
    print(chunk, end="", flush=True)

To set up a project for developing industrial applications, you will need a diverse team with a range of skills and expertise. Here is a proposed team composition based on the personas provided:

1. Project Manager (Manager Maya): As a globally recognized project manager with extensive experience in complex IT infrastructure projects, Manager Maya will be responsible for overseeing the project, ensuring it stays on track, within budget, and meets quality standards. Maya's expertise in coordinating cross-functional teams and problem-solving will be crucial for the project's success.

2. Product Manager (Manager Marco): With a decade of experience in the tech industry, specifically in Software as a Service (SaaS) products, Manager Marco will bring a holistic product perspective to the project. Marco's ability to align product features with user needs and business goals, as well as prioritize features using Agile methodologies, will be essential for developing industrial applications that

In [98]:
# stream response

for chunk in chain.stream(
"Can you advise with how to increase the number of units sold in a supermarket based off of summer sales and deals?"):
    print(chunk, end="", flush=True)

As a strategizing AI agent, I will provide insights on how to increase the number of units sold in a supermarket during the summer sales and deals period.

1. Social Media Manager (Manager Max): Utilize social media channels to promote summer sales and deals. Create engaging content that highlights special offers, discounts, and promotions to attract customers. Engage with followers and influencers to spread the word about the supermarket's summer offerings.

2. Hypothetical User (User Uma): Focus on enhancing the user experience during the summer sales and deals period. Ensure that the supermarket website and mobile app are user-friendly, intuitive, and provide value to customers. Collect feedback from users to continuously improve their shopping experience.

3. Product Manager (Manager Marco): Collaborate with the product team to introduce new products or variations that are suitable for summer sales and deals. Align product features with customer needs and preferences during the sum

In [99]:
# stream response

for chunk in chain.stream(
"What strategies should we consider to improve our financial performance?"):
    print(chunk, end="", flush=True)

To improve financial performance, you should consider the following strategies:

1. Utilize Advisor Amy's expertise in investment planning, retirement planning, estate planning, and risk management to assess and optimize your financial portfolio.

2. Seek guidance from Business Strategist Sophia to develop a strategic business plan that aligns with market dynamics, competitive landscape, and business models to drive growth and competitive advantage.

3. Leverage Manager Michael's HR strategies to create a positive work environment, foster employee engagement, and support employee growth and development, which can lead to increased productivity and performance.

4. Incorporate insights from the AI Prompt Engineer on how to navigate complexity, enhance adaptability, and foster a culture of continuous learning and innovation within your organization, which can contribute to long-term financial success.

By integrating the perspectives and expertise of these personas, you can develop a com

# Persona Selection

In [13]:
# List available Personas

personas = """

- Cybersecurity Expert
- DevOps Engineer
- Data Privacy Officer
- Physicist
- General Engineer
- Skeptic
- Optimist
- Historian
- Wise Guru
- Customer Service Representative
- Hypothetical User
- User Experience Designer
- Product Manager
- Project Manager
- Ethicist
- Business Strategist
- AI Researcher
- Frontend Developer
- Backend Developer
- Data Scientist
- Technical Writer
- Quality Assurance Specialist
- Threat Modeling Expert
- Organizational Cybernetician
- AI Prompt Engineer
- Ontologist & Semantic Architect
- Archaeologist
- Sociologist
- Cryptocurrency Analyst
- Environmental Scientist
- Human Resources Manager
- Financial Advisor
-Crisis Management Specialist
- Urban Planner
- Diversity, Equity, and Inclusion (DEI) Consultant
- Health Policy Analyst
- Social Media Manager
- Supply Chain Analyst
- Ethical Hacker
- Digital Marketing Specialist
- Occupational Therapist
- Renewable Energy Engineer
- Forensic Psychologist
- Fashion Designer
- Clinical Researcher
- Sports Physiotherapist
- Environmental Engineer
- Patent Attorney



"""

In [14]:
#Persona Selection prompt

system_prompt = """
Take into account the nuances and underlying themes of the following question, giving it careful consideration.

Question: {question}

"Carefully choose seven specialized personas from the provided list who possess the expertise required to 
establish a project aimed at developing industrial applications. Consider the individual depth and breadth 
of their skills, as well as the unique insights they offer. Recognize how their collective contributions may 
lead to innovative breakthroughs in addressing project challenges.".

I know you'll do great!

Available Personas: {personas}\n\n{format_instructions}

"""

In [15]:
# # Defines the response schema we want to receive

response_schemas = [
    ResponseSchema(name="persona1", description="the most relevant persona selected to use to reason through the question"),
    ResponseSchema(name="persona2", description="the second persona selected to use to reason through the question"),
    ResponseSchema(name="persona3", description="the third persona selected to use to reason through the question"),
    ResponseSchema(name="persona4", description="the fourth persona selected to use to reason through the question"),
    ResponseSchema(name="persona5", description="the fifth persona selected to use to reason through the question"),
    ResponseSchema(name="persona6", description="the sixth persona selected to use to reason through the question"),
    ResponseSchema(name="persona7", description="the seventh persona selected to use to reason through the question"),
    ResponseSchema(name="rationale", description="for each persona, carefully explain the most compelling reason for including them")

]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [16]:
# Instructions for how the output should be formatted into our initial prompt
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

In [17]:
model = ChatOpenAI(temperature=0.5)
chain = prompt | model | output_parser

In [ ]:
chain.invoke({"question": "Which experts have been selected for this project?"})

In [49]:
for chunk in chain.stream({
"question": "Which experts have been selected for this project?"}):
    print(chunk, end="", flush=True)

{'persona1': 'Data Scientist', 'persona2': 'Machine Learning Engineer', 'persona3': 'Subject Matter Expert', 'persona4': 'Project Manager', 'persona5': 'Software Developer', 'persona6': 'Data Engineer', 'persona7': 'User Experience Designer', 'rationale': {'Data Scientist': 'To analyze and interpret complex data sets to derive insights and make data-driven decisions.', 'Machine Learning Engineer': 'To design and implement machine learning algorithms to develop predictive models.', 'Subject Matter Expert': 'To provide domain-specific knowledge and insights to guide the project in the right direction.', 'Project Manager': 'To oversee the project, coordinate team members, and ensure timely delivery within budget.', 'Software Developer': 'To develop and maintain software applications, ensuring they meet the project requirements.', 'Data Engineer': 'To design, construct, install, test, and maintain highly scalable data management systems.', 'User Experience Designer': 'To focus on enhancing